# Results

In [32]:
import wandb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [35]:
def get_runs(entity, project): #(entity, project, log_keys=None, filters=None):
    api = wandb.Api(timeout=20)
    runs = api.runs(f"{entity}/{project}", filters={"created_at": {"$gt": "2025-02-23T20"}})
    print(f"Found {len(runs)} runs.")
    history = []
    for run in runs:
        #if log_keys:
        #    for log in run.scan_history(keys=log_keys):
        #        log["config"] = dict(run.config)
        #        history.append(log)
        #else:
        history.append({"config": dict(run.config), **dict(run.summary)})

    df = pd.json_normalize(history)
    return runs, df

In [36]:
runs, df = get_runs("jccamml","L65-project")

Found 32 runs.


In [37]:
df_full = df[df['_step'] > 8000]

In [27]:
[c for c in df_full.columns if "teacher" in c]

['config.hint_teacher_forcing']

In [15]:
df_full['config.algorithms']

0              [heapsort]
16       [insertion_sort]
17       [insertion_sort]
35       [insertion_sort]
36             [heapsort]
37             [heapsort]
38    [activity_selector]
39             [heapsort]
40       [insertion_sort]
41    [activity_selector]
42    [activity_selector]
43       [insertion_sort]
44             [heapsort]
45    [activity_selector]
46       [insertion_sort]
47             [heapsort]
48    [activity_selector]
49       [insertion_sort]
50             [heapsort]
51       [insertion_sort]
52    [activity_selector]
Name: config.algorithms, dtype: object

In [22]:
df_full['config.algorithms'] = df_full['config.algorithms'].apply(lambda x: x[0])   

/tmp/ipykernel_23478/205732895.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_full['config.algorithms'] = df_full['config.algorithms'].apply(lambda x: x[0])


In [46]:
df_full.pivot_table(index=['config.processor_type', 'config.hint_teacher_forcing', 'config.mpnn_processor_aggregator'], values=['insertion_sort_test_score', 'activity_selector_test_score'], aggfunc=np.mean)

/tmp/ipykernel_23478/2090578102.py:1: FutureWarning: The provided callable <function mean at 0x72afa8101da0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_full.pivot_table(index=['config.processor_type', 'config.hint_teacher_forcing', 'config.mpnn_processor_aggregator'], values=['insertion_sort_test_score', 'activity_selector_test_score'], aggfunc=np.mean)


activity_selector_test_score  \
config.processor_type      config.hint_teacher_forcing config.mpnn_processor_aggregator                                 
differential_mpnn          0.0                         max                                                   0.913444   
                                                       sum                                                   0.663537   
differential_mpnn2         0.0                         max                                                   0.956522   
                                                       sum                                                   0.821092   
differential_mpnn_identity 0.0                         max                                                   0.920578   
                                                       sum                                                   0.211495   
differential_mpnn_maxmax   0.0                         max                                                   0.889680   
differential_mpnn_msgdiff  0.0                         max                                                   0.935543   
mpnn                       0.0                         max                                                   0.942238   
                                                       sum                                                   0.649895   
                           0.5                         max                                                   0.956679   
                                                       sum                                                   0.238280   

                                                                                         insertion_sort_test_score  
config.processor_type      config.hint_teacher_forcing config.mpnn_processor_aggregator                             
differential_mpnn          0.0                         max                                                0.023926  
                                                       sum                                                0.869141  
differential_mpnn2         0.0                         max                                                0.291016  
                                                       sum                                                0.029297  
differential_mpnn_identity 0.0                         max                                                0.578125  
                                                       sum                                                0.128906  
differential_mpnn_maxmax   0.0                         max                                                0.581055  
differential_mpnn_msgdiff  0.0                         max                                                     NaN  
mpnn                       0.0                         max                                                0.617188  
                                                       sum                                                0.329102  
                           0.5                         max                                                0.149414  
                                                       sum                                                0.050293

In [ ]:
df_full.pivot_table(index=['config.processor_type', 'config.hint_teacher_forcing'], values=['test_score_current_model_heapsort', 'insertion_sort_test_score', 'activity_selector_test_score'], aggfunc=np.mean)

KeyError: 'test_score_current_model_heapsort'

In [24]:
for algorithm in df_full['config.algorithms'].unique():
    print(algorithm)
    print(df_full[df_full['config.algorithms'] == algorithm][['config.processor_type', f'test_score_current_model_{algorithm}']].to_markdown())
    print()

heapsort
|    | config.processor_type      |   test_score_current_model_heapsort |
|---:|:---------------------------|------------------------------------:|
|  0 | differential_mpnn          |                           0.706055  |
| 36 | mpnn                       |                           0.368164  |
| 37 | differential_mpnn2         |                           0.287598  |
| 39 | differential_mpnn_identity |                           0.0454102 |
| 44 | differential_mpnn          |                           0.382812  |
| 47 | mpnn                       |                           0.120117  |
| 50 | differential_mpnn_msgdiff  |                           0.45459   |

insertion_sort
|    | config.processor_type      |   test_score_current_model_insertion_sort |
|---:|:---------------------------|------------------------------------------:|
| 16 | differential_mpnn          |                                 0.675293  |
| 17 | mpnn                       |                                 0

In [31]:
import jax 
jax.devices()

[CpuDevice(id=0)]

In [48]:
np.expand_dims(np.zeros((3,4,2)),2).shape

(3, 4, 1, 2)

In [50]:
np.mean(np.zeros((5,6,7,8)), axis=1).shape

(5, 7, 8)

In [ ]:
def compute_entropy(logits):
    